In [1]:
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import numpy as np
import re
import urllib.request as req
warnings.filterwarnings('ignore')

In [2]:
# import full dataset
property_full = pd.read_csv('data/property-assessment-fy2018.csv')

In [13]:
# list of columns to save for properties
property_cols = ['ST_NUM','ST_NAME','ST_NAME_SUF','ZIPCODE','AV_TOTAL']

# drop all columns not in list (keep _ at end of name to show not fully clean yet)
property_ = property_full[property_full.columns[property_full.columns.isin(property_cols)]]

# drop rows with no street number because then the lat long is very off
property_df = property_[property_['ST_NUM'].str.strip().astype(bool)]

# drop rows with property price equal to zero because it will throw everything off
property_df = property_df[property_df['AV_TOTAL'] > 0]

In [24]:
# randomly sample 2500 rows to use on geocod.io free trial
geocode_df = property_df.sample(n=2499, replace=False, random_state=109)

In [21]:
for i,row in geocode_df.iterrows():
        
    # if zipcode doesnt exist still keep row bc lat long findable
    if np.isnan(row.ZIPCODE) == True: row.ZIPCODE = ' '
    else: row.ZIPCODE = str(row.ZIPCODE)[:-2]
        
    # convert everything to strings
    stnum   = str(row.ST_NUM)
    stname  = str(row.ST_NAME)
    stsuf   = str(row.ST_NAME_SUF)
    zipcode = str(row.ZIPCODE)
        
    # combine columns into one address
#     address = stnum.split(' ')[0] + '+' + stname + '+' + stsuf + '+' + zipcode + '+Boston,+MA,+USA'
    address = stnum.split(' ')[0] + ' ' + stname + ' ' + stsuf + ' ' + zipcode + ' Boston, MA, USA'
    geocode_df.at[i,'ADDRESS'] = address
    

In [25]:
# get df of only prices and adresses
addresses_df = geocode_df.drop(['ST_NUM','ST_NAME','ST_NAME_SUF','ZIPCODE'],axis=1)

In [23]:
# export to csv to give to geocod.io
addresses_df.to_csv('data/addresses_df.csv',header=['AV_TOTAL','ADDRESSES'])

In [29]:
# import geocoded dataset
geocoded = pd.read_csv('data/addresses_df_geocodio_8e9fd6b32bf419b7bcecf4f902184c8bdbe3e189.csv')

# drop all columns except for those we care about
property_sample = geocoded[['AV_TOTAL','Latitude','Longitude']]

# export to csv to use for model
property_sample.to_csv('data/property_sample.csv')

In [ ]:
%time
querie = '199+LEXINGTON+ST+2128+Boston+MA+USA'
# querie = '22+hillside+terrace,+belmont,+MA,+024780,+USA'
contents = req.urlopen('https://nominatim.openstreetmap.org/search?q='+querie+'&format=geocodejson').read()
longlat = re.findall(r'[\w\.-]+,[\w\.-]+', str(contents))

if bool(longlat): 
    long = longlat[0].split(',')[0]
    lat  = longlat[0].split(',')[1]
    
long, lat